In [44]:
#!/usr/bin/env python
import sys, os
sys.path.append("../v-rep_plugin") 
import numpy as np

## v-rep
import vrep

import matplotlib.pyplot as plt

from time import sleep
import math


In [54]:
def connect_vrep():
    print ('Program started')
    vrep.simxFinish(-1) # just in case, close all opened connections
    clientID = vrep.simxStart('127.0.0.1',19999,True,True,5000,5)
    if clientID != -1:
        print ('Connected to remote API server')
    else:
        print ('Failed connecting to remote API server')
    return clientID

In [55]:
def disconnect_vrep():
    vrep.simxFinish(clientID)
    print ('Program ended')

In [56]:
def call_sim_function(object_name, function_name, input_floats=[]):
    inputInts=[]
    inputFloats=input_floats
    inputStrings=[]
    inputBuffer=bytearray()
    res,retInts,retFloats,retStrings,retBuffer=vrep.simxCallScriptFunction(clientID,object_name,vrep.sim_scripttype_childscript,
                function_name,inputInts,inputFloats,inputStrings,inputBuffer,vrep.simx_opmode_blocking)

    return res,retInts,retFloats,retStrings,retBuffer
    
def get_laser_points():
    res,retInts,retFloats,retStrings,retBuffer = call_sim_function('LaserScanner_2D', 'get_laser_points')
    return retFloats

def get_global_path():
    res,retInts,retFloats,retStrings,retBuffer = call_sim_function('rwRobot', 'get_global_path')
    path_dist = []
    path_angle = []
    for i in range(len(retFloats)):
        if i < 2:
            continue
            
        if i%2 == 0:
            path_dist.append(retFloats[i])
        else:
            path_angle.append(retFloats[i])
    
    return path_dist, path_angle


In [100]:
def compute_reward(path_dist, reached_index):
    reward = -0.1
    closest_index = np.argmin(path_dist)
    if path_dist[closest_index] < 0.05 and reached_index < closest_index:
        reward = reward + 1
        reached_index = closest_index
    print closest_index, path_dist[closest_index]
    return reward, reached_index

In [121]:
def step(action, reached_index):
#     print 'old index: ', reached_index
    res,retInts,current_pose,retStrings,retBuffer = call_sim_function('rwRobot', 'step', action)
    laser_points = get_laser_points()
    path_dist, path_angle = get_global_path()
    
#     print path_dist
#     print path_angle

    reward, reached_index = compute_reward(path_dist, reached_index)
        
    is_finish = False

    return laser_points, current_pose, [path_dist, path_angle], reward, reached_index, is_finish

In [122]:
def test_callbadk():
    inputInts=[1,2,3]
    inputFloats=[53.21,17.39]
    inputStrings=['Hello','world!']
    inputBuffer=bytearray()
    res,retInts,retFloats,retStrings,retBuffer=vrep.simxCallScriptFunction(clientID,'rwRobot',vrep.sim_scripttype_childscript,
                    'callback',inputInts,inputFloats,inputStrings,inputBuffer,vrep.simx_opmode_blocking)
#     if res==vrep.simx_return_ok:
#         print (retInts)
#         print (retFloats)
#         print (retStrings)
#         print (retBuffer)
    

In [219]:
clientID = connect_vrep()
res,objs=vrep.simxGetObjects(clientID,vrep.sim_handle_all,vrep.simx_opmode_oneshot_wait)
if res == vrep.simx_return_ok:
        print ('Connected !! Number of objects in the scene: ', len(objs))
else:
        print ('Remote API function call returned with error code: ', res)

reached_index = -1
# laser_points = get_laser_points()
# path_dist, path_angle = get_global_path()

# print path_dist
# print path_angle

# plt.plot(path)
# plt.show()

# print len(path)

Program started
Connected to remote API server
('Connected !! Number of objects in the scene: ', 220)


In [228]:
new_laser, pose, path, reward, reached_index, is_finish = step([-1,-1,0,0,0], reached_index)
# print 'new index: ', reached_index
print reward

3 0.0251347888261
-0.1


In [218]:
disconnect_vrep()

Program ended
